## ASG5 part3
          

Cluster the business sections (/blue/acg7849/share/BS) using Doc2Vec (50 clusters) in two ways:

Using a counter as the ‘tag’ (as in 5.1.5)

Using a counter as the ‘tag’, and the industry code as an additional tag (yield TaggedDocument(words=file_tokens, tags=[i, SIC]) where SIC is a string holding the tag (for example ‘1740’)

Extract the 4-digit SIC industry code from the annual report header (STANDARD INDUSTRIAL CLASSIFICATION).

Required: Evaluate whether adding the industry code as an additional tag improves the clustering. Use the standard deviation of profitability as a way to evaluate this. (Firms that are more similar, should have similar performance. Therefore, a better clustering would result in lower standard deviations for each cluster, relative to a worse clustering).

Do this for the filings for the year 2019 only. Calculate the standard deviation of performance for each cluster (use the year of CONFORMED END OF PERIOD, which are the first 4 digits of ‘date’ in summary.text).

For 50 clusters that means you will have 2 standard deviations for each cluster (one for each approach, with the extra SIC tag vs not adding the extra SIC tag). Use a t-test to test for a difference between the two sets of 50 standard deviations.

In [1]:
# all imports
import os as os
import pandas as pd
import glob
import csv
from pathlib import Path
import html, re
from w3lib.html import replace_entities
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from scipy.cluster import  hierarchy
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
import gensim
stopWords = set(stopwords.words('english') )
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk import FreqDist
from nltk.collocations import *

/apps/jupyterhub/1.1.0/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# already made this - copy + paste to personal directory

#try copy and past targets
#think this is a pretty bold way avoiding messing up with the class

import shutil
for i in range(len(listoflink)):
    shutil.copy(listoflink[i], r'/blue/acg7849/hli1/item1')
    #shutil.copy(listoflink[i], r'/blue/acg7849/tbode/whateverfile')

# codes from 515 - start with new directory

In [2]:
# add some punctuation to string.punctuation
punc = string.punctuation + '“”'

# documents get tagged by an index (number), while filenames have different numbers
# keep track of this
fileIdToIndex = {} # given a fileId -> tag
indexToFileId=[] # given a tag -> fileId

class MyFiles(object):
    def __init__(self, dirname, tokens_only = False):
        self.dirname = dirname
        self.tokens_only = tokens_only
 
    def __iter__(self):
        #for i, fname in enumerate(files[0:200]):
         
        for i, fname in enumerate(os.listdir(self.dirname)[0:200]):    # there are 4604 files in total
        # enumerate = return a list of tuples, iterate from start to end
        # os.listdir = return index of a directory, input = directory address
        # this part enumerates the first 200 units in the index under dirname
        
            with open( os.path.join(self.dirname, fname), encoding='utf-8') as f:
                content = f.read()
            # filter
                #content = [f for f in content if int(f["length"])> 1000 and (f["date"][0:4]) == '2019']
                #content = [f for f in content if len(f)> 1000]
            # grab id from filename
            myCounter = int ( re.findall(r'(\d*)\.txt', fname)[0] )
            # update 
            fileIdToIndex [ myCounter] = i
            indexToFileId.append( myCounter)
            #print('fname', fname, 'tag', myCounter)
            file_tokens = [x for x in word_tokenize(content) if x.isalpha() and x.lower() not in stopWords and x not in string.punctuation]
            
            if self.tokens_only == True:
                yield file_tokens
            else:
                yield TaggedDocument(words=file_tokens, tags=[i] )                    

In [3]:
# Hipergator
ffiles = MyFiles(r'/blue/acg7849/hli1/item1') # this one expected str, bytes or os.PathLike object
#ffiles = MyFiles(r'/blue/acg7849/share/BS/item1/') # a memory-friendly iterator
# dirname = '/blue/acg7849/share/BS/item1/'?

In [4]:
# create a model, build vocabulary
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=8)
model.build_vocab(ffiles) 

In [5]:
# train it
model.train(ffiles, total_examples=model.corpus_count, epochs=model.epochs)
print('FINISH')

FINISH


In [6]:
# Hipergator
def tokenizeFile(file_id):
    with open( r'/blue/acg7849/hli1/item1/'+str(file_id)+'.txt', encoding='utf-8') as f:
            content = f.read()
    return ([x for x in word_tokenize(content) if x.isalpha() and x.lower() not in stopWords and x not in string.punctuation] )

In [7]:
t = tokenizeFile(277350) # test file = 277350
model.infer_vector( t )

array([-1.4767705 , -0.0115789 , -1.7262841 , -1.2262379 ,  2.7107096 ,
       -1.9205048 , -2.519832  , -1.2013861 , -2.6640322 ,  0.6726109 ,
       -0.26111737,  2.4964356 , -0.9154145 ,  1.1359534 ,  0.39194626,
        0.40125725,  0.6422191 ,  2.1713967 , -1.7170639 , -4.838732  ,
       -0.6639528 ,  0.3315963 ,  0.13589436,  0.36715427, -0.19252302,
       -0.42660287, -1.3432615 ,  0.23694478, -1.6745603 ,  0.94445527,
        0.80584824,  1.6799648 , -0.938454  , -1.531435  ,  0.79121536,
       -1.4815254 , -0.2635625 , -2.859668  ,  1.7009642 ,  0.3152727 ,
       -0.7396305 , -0.62165445,  1.823005  ,  1.5146507 ,  0.10767917,
       -2.9715924 ,  0.37576294,  2.1433923 ,  0.46156064,  0.37237906,
       -1.0037848 ,  1.487787  ,  0.89794105,  0.5859917 , -2.128481  ,
       -0.28554684, -1.5388486 ,  1.994815  , -0.8447659 ,  1.6837491 ,
        3.272897  ,  0.42764863,  2.0898871 , -1.1646743 , -1.0380759 ,
       -0.9085714 ,  1.0646596 ,  0.5012265 ,  0.31648484,  0.26

In [8]:
t = tokenizeFile(277350)
inferred_vector = model.infer_vector( t )
# dv is short for docvecs
# most similar files to 277350 is the 4th one 265065?
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
#sims = model.dv.most_similar([inferred_vector], topn=4)
sims[0:5]

[(180, 0.7388142943382263),
 (3, 0.7200323939323425),
 (107, 0.6929206252098083),
 (26, 0.6576422452926636),
 (37, 0.6475551128387451)]

In [9]:
# letter with filename 1.txt is the first letter, so tag is 0
similar_doc = model.docvecs.most_similar(0)
similar_doc

<ipython-input-9-0bceb90a8426>:2: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_doc = model.docvecs.most_similar(0)


[(103, 0.8445403575897217),
 (93, 0.8397471904754639),
 (149, 0.8357316851615906),
 (107, 0.8090888857841492),
 (198, 0.7986845374107361),
 (55, 0.7941851615905762),
 (115, 0.7887516021728516),
 (90, 0.7636479735374451),
 (47, 0.7595515251159668),
 (37, 0.7589263916015625)]

In [10]:
print('number of documents', model.corpus_count)
print('model.docvecs', len(model.dv))

number of documents 200
model.docvecs 200


In [11]:
# reread the files, and get the vector for each file
# feed vector into k-means algorithm to make clusters
wordLists = MyFiles(r'/blue/acg7849/hli1/item1', tokens_only = True) # a memory-friendly iterator
vectors = [ model.infer_vector( w ) for w in wordLists]
print(len(vectors))
vectors[0]

200


array([ 0.28004587,  0.18462417, -1.1665092 , -0.2097246 ,  2.5319278 ,
       -0.41135722, -4.112002  , -0.16554652, -3.097385  , -2.0491154 ,
        1.3159641 ,  2.7108033 , -0.6671405 ,  0.15734433, -1.3450367 ,
        0.66420406,  2.7075136 ,  2.5028274 , -2.5337396 , -2.4513004 ,
       -3.8098    , -0.44515482,  0.06447091, -0.9705248 ,  1.2377226 ,
       -3.1624103 , -2.044037  ,  1.6543127 , -0.27113047, -0.93310666,
        3.1243086 ,  0.97415537, -1.0976663 , -0.45981872, -0.703838  ,
       -1.9956176 ,  0.5954963 , -4.513073  ,  1.0142688 , -0.83850384,
       -0.64833117, -1.7494752 ,  2.8035305 , -0.23724718, -1.4981656 ,
       -4.303735  ,  0.0534537 ,  0.38065517,  2.165428  ,  1.1070954 ,
        2.519425  ,  1.8053597 ,  0.48967382, -0.10304538, -2.3480372 ,
        2.699306  , -0.9138088 ,  1.0819453 ,  1.4957076 ,  4.373846  ,
        3.8091147 ,  0.257506  ,  4.848016  ,  0.2955171 , -0.69672257,
       -1.0947207 , -1.3238903 ,  0.62429506,  0.5751652 , -0.58

In [12]:
from nltk.cluster import KMeansClusterer
num_clusters = 20
kclusterer = KMeansClusterer(num_clusters, distance=nltk.cluster.util.cosine_distance, repeats=25)

#assigned_clusters: sequence of files and matching clusters
assigned_clusters = kclusterer.cluster(vectors, assign_clusters=True)
assigned_clusters[0:10]

[12, 19, 17, 9, 10, 5, 4, 4, 17, 2]

In [13]:
import collections
print(collections.Counter(assigned_clusters))
# give length of each cluster

Counter({16: 27, 13: 18, 9: 17, 2: 16, 14: 15, 4: 14, 15: 13, 0: 11, 12: 10, 5: 9, 18: 9, 19: 7, 11: 7, 10: 6, 17: 5, 6: 5, 8: 5, 1: 3, 3: 2, 7: 1})


# add performance and do sd of performance of each cluster

In [14]:
pfm = pd.read_csv (r'performance.csv')  
pfm = pfm.loc[pfm["year"] == 2019]
pfm[0:5] # OK
#backup new csv
#pfm.to_csv('pfm19.csv', index = False)

,cik,performance,year
14470,1750,0.011929,2019
14471,6201,0.028102,2019
14472,3197,0.043332,2019
14473,1230869,0.319006,2019
14474,764622,0.029131,2019


In [15]:
# need: left join (asg5 csv, pfm19)
asg5 = pd.read_csv (r'ASG5.csv') 
pfm = pd.read_csv (r'pfm19.csv') 
pfm = pfm.rename(columns={'cik': 'CIK'})
pfm[0:5]

,CIK,performance,year
0,1750,0.011929,2019
1,6201,0.028102,2019
2,3197,0.043332,2019
3,1230869,0.319006,2019
4,764622,0.029131,2019


In [16]:
table1 = asg5.join(pfm.set_index('CIK'), on='CIK')
#df.join(other.set_index('key'), on='key')

In [17]:
# drop last column
table1 = table1.iloc[: , :-1]  

In [18]:
# need to drop all NA on performance later
#table1.to_csv('table1.csv', index = False)

# build a df/csv for 200 files + 20 clusters (test)

In [19]:
table1_test = table1[0:200]

In [20]:
#table1_test.to_csv('table1_test.csv', index = False)

In [21]:
len(assigned_clusters)

200

In [22]:
table1_test['clstr'] = assigned_clusters

<ipython-input-22-bb889d3a5fdf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table1_test['clstr'] = assigned_clusters


,CIK,coName,formtype,date,fName,length,full_link,performance,clstr
0,1345016,YELP INC,10-K,2019-12-31,280603.txt,152323,/blue/acg7849/share/BS/item1/280603.txt,0.038182,12
1,1772177,"KURA SUSHI USA, INC.",10-K,2019-08-31,275323.txt,38468,/blue/acg7849/share/BS/item1/275323.txt,0.019055,19
2,1341766,"Celsius Holdings, Inc.",10-K,2019-12-31,280595.txt,132197,/blue/acg7849/share/BS/item1/280595.txt,0.110321,17
3,1679826,Ping Identity Holding Corp.,10-K,2019-12-31,283125.txt,100202,/blue/acg7849/share/BS/item1/283125.txt,-0.001723,9
4,1650445,Quorum Health Corp,10-K,2019-12-31,282859.txt,146425,/blue/acg7849/share/BS/item1/282859.txt,-0.109879,10
5,49600,EASTGROUP PROPERTIES INC,10-K,2019-12-31,284462.txt,20617,/blue/acg7849/share/BS/item1/284462.txt,0.047784,5
6,1174891,"CalEthos, Inc.",10-K,2019-12-31,280098.txt,88622,/blue/acg7849/share/BS/item1/280098.txt,NaN,4
7,1693696,UNITED CAPITAL CONSULTANTS INC.,10-K,2019-12-31,283268.txt,90266,/blue/acg7849/share/BS/item1/283268.txt,NaN,4
8,1630176,GREY CLOAK TECH INC.,10-K,2019-12-31,282682.txt,33461,/blue/acg7849/share/BS/item1/282682.txt,NaN,17
9,83350,RESERVE PETROLEUM CO,10-K,2019-12-31,285179.txt,23514,/blue/acg7849/share/BS/item1/285179.txt,-0.007474,2


In [26]:
table1_test = table1_test.dropna()
len(table1_test)

164

In [28]:
table1_test[0:5]

,CIK,coName,formtype,date,fName,length,full_link,performance,clstr
0,1345016,YELP INC,10-K,2019-12-31,280603.txt,152323,/blue/acg7849/share/BS/item1/280603.txt,0.038182,12
1,1772177,"KURA SUSHI USA, INC.",10-K,2019-08-31,275323.txt,38468,/blue/acg7849/share/BS/item1/275323.txt,0.019055,19
2,1341766,"Celsius Holdings, Inc.",10-K,2019-12-31,280595.txt,132197,/blue/acg7849/share/BS/item1/280595.txt,0.110321,17
3,1679826,Ping Identity Holding Corp.,10-K,2019-12-31,283125.txt,100202,/blue/acg7849/share/BS/item1/283125.txt,-0.001723,9
4,1650445,Quorum Health Corp,10-K,2019-12-31,282859.txt,146425,/blue/acg7849/share/BS/item1/282859.txt,-0.109879,10


In [45]:
# calculate sd(perf) by clstr
#table1_test.sql('SELECT performance,clstr, sum(performance) GROUP BY clstr')
table1_test_sd = table1_test.groupby('clstr').std()
table1_test_sd = table1_test_sd ['performance']# we get sd of performance
print(table1_test_sd)

clstr
0     0.002738
1     0.005545
2     0.922740
3     0.013788
4     0.130794
5     0.057892
6     0.015134
7          NaN
8     0.058374
9     0.388929
10    0.102689
11    6.329008
12    0.180032
13    3.432318
14    0.114915
15    0.182796
16    1.093205
17    1.078114
18    0.123319
19    0.046373
Name: performance, dtype: float64
